In [1]:
# move this down
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel 
ds = pd.read_csv('scotch_review.csv')

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pickle
import csv
import time

In [26]:
# Finds list of whiskey names

URL1 = 'https://scotchnoob.com/reviews/?pagenum=1&dateFrom=11/10/2010&dateTo=04/11/2021&shownum=-1&orderBy=title&order=ASC&priceFrom=0&priceTo=-1&region=All&rating=All&tag_peated=false&tag_sherried=false&tag_indie=false'

page1 = requests.get(URL1)

soup1 = BeautifulSoup(page1.text, 'lxml')


In [128]:
# Finds whiskey names
whiskey_names = []

name0 = soup1.find('div', {'id' : 'filter_results'})

whiskey_names_old = name0.find_all('h4')
for w in range(len(whiskey_names_old)):
    
    whiskey_names[w] = whiskey_names[w].text



# Finds whiskey urls

whiskey_urls = []

for tag in name0.find_all('a', href = True):
    whiskey_urls.append(str(tag['href']))





whiskey_desc = []


for r in range(1):
    
    URL0 = whiskey_urls[r]
    
    page0 = requests.get(URL0)

    soup0 = BeautifulSoup(page0.text, 'lxml')
    
    # Finds description of whiskey
    
    # Builds temp array to join and append to whiskey_desc array
    class0 = soup0.find('div', {'class' : 'entry-content'})
    class1 = class0.find_all('strong')
    
   
    whiskey_desc_mass = []
    for e in class1:
        whiskey_desc_mass.append(e.next_sibling)
    
    [''.join(whiskey_desc_mass) for w in whiskey_desc_mass]
    
    whiskey_desc.append(whiskey_desc_mass)
    
    
    
print(whiskey_desc[0])  

IndexError: list index out of range

In [68]:
print(whiskey_urls[0])
print(whiskey_names[0])

https://scotchnoob.com/2016/10/10/1792-port-finish-bourbon/
1792 Port Finish Bourbon 


In [69]:
URL0 = whiskey_urls[0]

page0 = requests.get(URL0)

soup0 = BeautifulSoup(page0.text, 'lxml')

In [71]:
# Finds description of whiskey

class0 = soup0.find('div', {'class' : 'entry-content'})
class1 = class0.find_all('strong')
class2 = class0.find_all('p')

for e in class1:
    print(e.next_sibling)

: Grape juice and grape jam, on a bed of sweet corn and caramel. Quite sweet. After a rest in the glass, a tart top note – like cranberry – creeps in.
: A tad hot for ~44%. 
: Short. Oaky, but not tannic. Echoes of sweet grape jam and semisweet chocolate. Fades quickly.
: Water brings a little more nose tickle to the aroma, without revealing anything new. The palate is a little softer. I suggest adding a touch of water and then letting it rest in the glass for a few minutes to reawaken the aroma.
: Not quite as supple and polished as 


In [113]:
print(whiskey_desc)

[]


In [127]:
whiskey_desc = []


for r in range(1):
    
    URL0 = whiskey_urls[r]
    
    page0 = requests.get(URL0)

    soup0 = BeautifulSoup(page0.text, 'lxml')
    
    # Finds description of whiskey
    
    # Builds temp array to join and append to whiskey_desc array
    class0 = soup0.find('div', {'class' : 'entry-content'})
    class1 = class0.find_all('strong')
    
   
    whiskey_desc_mass = []
    for e in class1:
        whiskey_desc_mass.append(e.next_sibling)
    
    [''.join(whiskey_desc_mass) for w in whiskey_desc_mass]
    
    whiskey_desc.append(whiskey_desc_mass)
    
    
    
print(whiskey_desc[0])    




[': Grape juice and grape jam, on a bed of sweet corn and caramel. Quite sweet. After a rest in the glass, a tart top note – like cranberry – creeps in.', ': A tad hot for ~44%. ', ': Short. Oaky, but not tannic. Echoes of sweet grape jam and semisweet chocolate. Fades quickly.', ': Water brings a little more nose tickle to the aroma, without revealing anything new. The palate is a little softer. I suggest adding a touch of water and then letting it rest in the glass for a few minutes to reawaken the aroma.', ': Not quite as supple and polished as ']
